## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Pueblo Rico,CO,5.24,-76.04,77.00,65,40,6.91,scattered clouds
1,1,Tasiilaq,GL,65.61,-37.64,19.40,86,90,23.02,snow
2,2,Punta Arenas,CL,-53.15,-70.92,44.60,65,0,23.02,clear sky
3,3,Namibe,AO,-15.20,12.15,78.85,82,88,5.44,overcast clouds
4,4,Ust-Tsilma,RU,65.44,52.15,17.87,98,50,3.65,scattered clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input('What is the minimum temperature you would like for your trip? '))
max_temp = float(input('What is the maximum temperature you would like for your trip? '))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 100


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & \
                                       (city_data_df['Max Temp'] >= min_temp)]
preferred_cities_df.head(5)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Pueblo Rico,CO,5.24,-76.04,77.00,65,40,6.91,scattered clouds
3,3,Namibe,AO,-15.20,12.15,78.85,82,88,5.44,overcast clouds
5,5,Cap Malheureux,MU,-19.98,57.61,81.00,75,8,8.01,clear sky
6,6,Saint George,US,37.10,-113.58,89.01,9,1,4.61,clear sky
8,8,Olinda,BR,-8.01,-34.86,80.60,74,75,5.75,broken clouds


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                6
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.head(5)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Pueblo Rico,CO,5.24,-76.04,77.00,65,40,6.91,scattered clouds
3,3,Namibe,AO,-15.20,12.15,78.85,82,88,5.44,overcast clouds
5,5,Cap Malheureux,MU,-19.98,57.61,81.00,75,8,8.01,clear sky
6,6,Saint George,US,37.10,-113.58,89.01,9,1,4.61,clear sky
8,8,Olinda,BR,-8.01,-34.86,80.60,74,75,5.75,broken clouds


In [12]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(5)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Pueblo Rico,CO,77.00,scattered clouds,5.24,-76.04,
3,Namibe,AO,78.85,overcast clouds,-15.20,12.15,
5,Cap Malheureux,MU,81.00,clear sky,-19.98,57.61,
6,Saint George,US,89.01,clear sky,37.10,-113.58,
8,Olinda,BR,80.60,broken clouds,-8.01,-34.86,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found... skipping.")    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [14]:
# 7. Drop the rows where there is no Hotel Name.

# 7.a) Replace empty rows with nAn
hotel_df.replace('', np.nan, inplace=True)

# 7.b) Drop any row with nAn
clean_hotel_df = hotel_df.dropna()

In [15]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [23]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [24]:
# 11a. Add a marker layer for each city to the map. 
# max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
#              max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))